## Reformat original json files

In [36]:
import json

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020/fall2020(jan2).json", "r") as read_file:
    vergil = json.load(read_file)
    read_file.close()

#put all 0000-4000 level courses in 'undergrad' array
undergrad = []
for course in vergil:
    courseId = course["course"]["course_identifier"]
    
    if len(courseId) == 9 or len(courseId) == 10:
        if courseId[-4].isdigit():
            if int(courseId[-4]) <= 4:
                #print(courseId)
                undergrad.append(course["course"])
                
#print(len(undergrad)) 
#print(undergrad[2:3])

#extract each course section's time and max enrollment
alltimeDicts = {}
types = []
for course in undergrad:
    courseId = course["course_identifier"]
    
    sections = course["classes"]["class"]
    times = []
    info = {}
    
    
    for section in sections:
        timesListed = section["days_times"]
        size = section["enrollment"]["max"]
        classtype = section["type"]["type_code"]
        classtypename = section["type"]["name"]
        
        #print(classtype, classtypename)
        
        for day in timesListed:
            startT = day["mil_time_from"]
            endT = day["mil_time_to"]
            time = [startT, endT]
            if startT and endT:
                times.append(time)
                if classtypename not in types:
                    types.append(classtypename)
    
    
    if times:
        info.update({"size": size})
        info.update({"times": times})
        info.update({"type": classtypename})
        
    if info:
        alltimeDicts.update({courseId: info})

for type in types:
    print(type)

#print(len(alltimeDicts))
#print(alltimeDicts)

#write to json
'''
with open("spring2021ugradData.json","w") as write:
    json.dump(alltimeDicts, write, indent = 4)
'''

Lecture
Recitation
Seminar
Studio
Independent
Discussion
Laboratory
Conversation
Language
Rehearsal
Field Work
Workshop
Practica
Research Sem
Oral/Aural P
Colloquia
Tutorial
Physical Edu
Online Cours
Registration


'\nwith open("spring2021ugradData.json","w") as write:\n    json.dump(alltimeDicts, write, indent = 4)\n'

## Convert time zones

In [7]:
from datetime import datetime, timezone
import pytz
from parse import *
import json

with open("spring2020ugradData.json", "r") as read_file:
    ugrad = json.load(read_file)
    read_file.close()
    
# for tz in pytz.all_timezones:
#     print(tz)

#set standard date as 01-01-2020 
#not during daylight savings, so 23 hrs between nyc and beijing instead of 24
standard = "01-01-2020 "

fmt = "%H:%M"

#timezone variables
ny = pytz.timezone('America/New_York')
bj = pytz.timezone('Asia/Shanghai')
la = pytz.timezone('America/Los_Angeles')

def convert(targetZone, timeStr):
    dt = datetime.strptime(timeStr, "%d-%m-%Y %H:%M:%S")
    dt = ny.localize(dt)
    dtconverted = dt.astimezone(targetZone)
    return dtconverted.strftime(fmt)

bjArr = ugrad
bjStartArr = []
for course in ugrad:
    for time in ugrad[course]["times"]:
        start = time[0]
        start = standard + start + ":00"
        bjArr[course]["times"][0][0] = convert(bj, start)

        end = time[1]
        end = standard + end + ":00"
        bjArr[course]["times"][0][1] = convert(bj, end)
        
    bjStartArr.append(convert(bj, start))

#print(bjArr)
#print(bjStartArr)

with open("spring2020ugrad_bj.json","w") as write:
    json.dump(bjArr, write, indent = 4)

## Check class sizes

In [39]:
import json
import statistics as stats

with open("C:/Users/Administrator/Desktop/spec/vergil_wrangle/fall2020/fall2020ugradData.json", "r") as read_file:
    courses = json.load(read_file)
    read_file.close()
    
size = []    
nullCount = 0
for course in courses:
    if not courses[course]["size"]:
        #print(course)
        nullCount+=1
    else:
        if courses[course]["type"] == "Studio":
            print(course , courses[course]["size"])
            size.append(int(courses[course]["size"]))
        
print(stats.mean(size))

DNCEBC2250 20
DNCEBC2139 23
DNCEBC2137 23
DNCEBC1332 25
DNCEBC1330 35
DNCEBC1250 20
DNCEBC1247 20
DNCEBC3601 15
DNCEBC3335 25
DNCEBC3332 25
DNCEBC3250 15
DNCEBC3249 25
DNCEBC3140 25
DNCEBC1137 25
DNCEBC3138 25
DNCEBC2452 25
DNCEBC2334 25
DNCEBC2332 25
DNCEBC2255 40
DNCEBC2253 25
DNCEBC2252 30
DNCEBC1135 23
AHISBC3530 999
AHISBC3003 18
VIARR1700 13
MUSIW4630 15
ARCHA4005 999
ARCHA4510 999
ARCHA4003 999
ARCHA4001 999
PNTGK0100 15
DRAWK0100 15
ARCHV3201 100
ARCHV2103 100
ARCHV2101 100
ARCHW1020 100
THTRV3405 999
THTRV3300 999
THTRV3200 999
230.94871794871796


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection




In [196]:
import xlwt 
from xlwt import Workbook 
  
# Workbook is created 
wb = Workbook() 
  
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1') 

i = 0
for starttime in bjStartArr:
    sheet1.write(i, 0, starttime)
    i+=1
  
wb.save('Desktop/spec/vergil_wrangle/bjStartTime.csv') 

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/spec/vergil_wrangle/bjStartTime.csv'